In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nb
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform
import pickle
import pandas as pd
from matplotlib.colors import Normalize

In [2]:
cwd = os.getcwd()
analysis_result_dir = Path(cwd).parent / "data"
figure_result_dir = Path(cwd).parent / "figures"
print('Running on {}'.format(platform.node()))
print('Current deriv folder is {}'.format(analysis_result_dir))

print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

Running on Smoldering-Corpse-Bar.local
Current deriv folder is /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/pRF_expect_analysis/prf_expect/analysis/data
cortex.database.default_filestore: /Users/dionysus/Documents/pycortex/db
cortex.options.usercfg: /Users/dionysus/Library/Application Support/pycortex/options.cfg


In [3]:
analysis_result_dir = Path(analysis_result_dir)
subject = "fsaverage-vis"

In [4]:
r2_thre = 0.15
surprise_analysis_type = 'TRMI-type1'

### Run vis_TRMI_on_visual_hierarchy.ipynb first, it will generate the TRMI_fn needed below.

In [5]:
# polar angle and prf mapping stat maps
TRMI_fn = os.path.join(analysis_result_dir, f"group_analysis-{surprise_analysis_type}_space-fsaverage_TRMI.npy")
rsq_fn = os.path.join(analysis_result_dir, "group_analysis-pRF_space-fsaverage_rsq.npy")
viol_tstat_fn = os.path.join(analysis_result_dir, f"group_analysis-{surprise_analysis_type}_space-fsaverage_viol_tstat.npy")
TRMI = np.load(TRMI_fn, allow_pickle=True)
param_rsq = np.load(rsq_fn, allow_pickle=True)
viol_tstat = np.load(viol_tstat_fn, allow_pickle=True)
TRMI[np.isnan(TRMI)] = 0
param_rsq[np.isnan(param_rsq)] = 0
viol_tstat[np.isnan(viol_tstat)] = 0

param_B_fn = os.path.join(analysis_result_dir, "group_analysis-pRF_space-fsaverage_b.npy")
param_B = np.load(param_B_fn, allow_pickle=True)
param_B[np.isnan(param_B)] = 0

param_D_fn = os.path.join(analysis_result_dir, "group_analysis-pRF_space-fsaverage_d.npy")
param_D = np.load(param_D_fn, allow_pickle=True)
param_D[np.isnan(param_D)] = 0

param_pa_fn = os.path.join(analysis_result_dir, "group_analysis-pRF_space-fsaverage_pa.npy")
param_pa = np.load(param_pa_fn, allow_pickle=True)
param_pa[np.isnan(param_pa)] = 0

param_ecc_fn = os.path.join(analysis_result_dir, "group_analysis-pRF_space-fsaverage_ecc.npy")
param_ecc = np.load(param_ecc_fn, allow_pickle=True)
param_ecc[np.isnan(param_ecc)] = 0

param_prf_size_fn = os.path.join(analysis_result_dir, "group_analysis-pRF_space-fsaverage_prf_size.npy")
param_prf_size = np.load(param_prf_size_fn, allow_pickle=True)
param_prf_size[np.isnan(param_prf_size)] = 0


In [6]:
static_imgs = False
web_view = True

In [7]:
def con_weighted_dispcx(subject, data, param_rsq, cmap='seismic', vmin=-0.3, vmax=0.3, vmin2=0, vmax2=1, rsq_thre=0.25):
    curv = cortex.db.get_surfinfo(subject)
    # Adjust curvature contrast / color. Alternately, you could work
    # with curv.data, maybe threshold it, and apply a color map. 
    curv.vmin = -1
    curv.vmax = 1
    curv.cmap = 'gray'
    curv.data = curv.data * .75 + 0.6
    # curv.data = curv.data * .50
    norm2 = Normalize(vmin2, vmax2)
    # curv = cortex.Vertex(curv.data, subject, vmin=-1,vmax=1,cmap='gray')
    # Create some display data

    # # normalize the range of param_rsq to 0 to 1
    # if param_rsq.max()-param_rsq.min()>0:
    #     param_rsq = (param_rsq-param_rsq.min())/(param_rsq.max()-param_rsq.min()-0.2)
    # else:
    #     pass
    # alpha = np.clip(norm2(param_rsq), 0, 1)
    alpha = np.clip(norm2(param_rsq), 1, 1)
    alpha[param_rsq<rsq_thre] = 0
    vx = cortex.Vertex(data, subject, cmap=cmap, vmin=vmin, vmax=vmax, )

    # Map to RGB
    vx_rgb = np.vstack([vx.raw.red.data, vx.raw.green.data, vx.raw.blue.data])
    curv_rgb = np.vstack([curv.raw.red.data, curv.raw.green.data, curv.raw.blue.data])

    
    # alpha = param_rsq
    alpha = alpha.astype(np.float32)

    # Alpha mask
    display_data = vx_rgb * alpha + curv_rgb * (1 - alpha)
    # fake_curv_rgb = np.zeros_like(curv_rgb)
    # display_data = vx_rgb * alpha + fake_curv_rgb * (1 - alpha)
    # display_data /= 255
    return display_data

In [8]:
def Vertex2D_fix(data1, data2, subject, cmap, vmin, vmax, vmin2, vmax2, roi_borders=None):
    #this provides a nice workaround for pycortex opacity issues, at the cost of interactivity    
    # Get curvature
    curv = cortex.db.get_surfinfo(subject)
    # Adjust curvature contrast / color. Alternately, you could work
    # with curv.data, maybe threshold it, and apply a color map. 
    
    #standard
    curv.data = curv.data * .75 +0.1
    #alternative
    #curv.data = np.sign(curv.data) * .25
    #HCP adjustment
    #curv.data = curv.data * -2.5# 1.25 +0.1

    
    curv = cortex.Vertex(curv.data, subject, vmin=-1,vmax=1,cmap='gray')
    
    norm2 = Normalize(vmin2, vmax2)   
    
    vx = cortex.Vertex(data1, subject, cmap=cmap, vmin=vmin, vmax=vmax)
    
    # Map to RGB
    vx_rgb = np.vstack([vx.raw.red.data, vx.raw.green.data, vx.raw.blue.data])
    
    curv_rgb = np.vstack([curv.raw.red.data, curv.raw.green.data, curv.raw.blue.data])

    
    # Pick an arbitrary region to mask out
    # (in your case you could use np.isnan on your data in similar fashion)
    alpha = np.clip(norm2(data2), 0, 1)

    # Alpha mask
    display_data = (curv_rgb * (1-alpha)) + vx_rgb * alpha

    display_data /= 255


    #print(display_data.min())
    #print(display_data.max())
    
    if roi_borders is not None:
        display_data[:,roi_borders.astype('bool')] = 0#255-display_data[:,roi_borders.astype('bool')]#0#255
    
    # Create vertex RGB object out of R, G, B channels
    return cortex.VertexRGB(*display_data, subject)  

In [9]:
rsq_alpha = np.ones_like(param_rsq)
rsq_alpha[param_rsq<0.1] = 0

display_rsq = con_weighted_dispcx(
    subject, 
    param_rsq, 
    rsq_alpha, 
    cmap='inferno', 
    vmin=0.1, 
    vmax=0.7,
    rsq_thre=0.1
)
rsq_cx = cortex.VertexRGB(
    *display_rsq, 
    subject,
)

# rsq_cx = cortex.Vertex(param_rsq, subject, vmin=0.1, vmax=0.7, cmap='inferno')

display_TRMI = con_weighted_dispcx(
    subject, 
    TRMI, 
    param_rsq, 
    cmap='BrBG_r',
    vmin=-0.15, 
    vmax=0.15,
    vmin2=0.1,
    vmax2=0.5
)
cx_TRMI = cortex.VertexRGB(
    *display_TRMI, 
    subject,
)

display_viol_tstat = con_weighted_dispcx(
    subject, 
    viol_tstat, 
    param_rsq, 
    cmap='BrBG_r',
    vmin=-2, 
    vmax=2,
    vmin2=0.1,
    vmax2=0.5
)
cx_viol_tstat = cortex.VertexRGB(
    *display_viol_tstat, 
    subject,
)


rsq_alpha = np.ones_like(param_rsq)
rsq_alpha[param_rsq<0.1] = 0
display_sub_r2 = con_weighted_dispcx(
    subject, 
    np.array(param_rsq), 
    rsq_alpha, 
    cmap='inferno', 
    vmin=0.1, 
    vmax=0.8
)
cx_sub_r2 = cortex.VertexRGB(
    *display_sub_r2, 
    subject, 
)

display_ecc = con_weighted_dispcx(
    subject, 
    np.array(param_ecc), 
    np.array(param_rsq), 
    cmap='Spectral', 
    vmin=0, 
    vmax=5
)
cx_ecc = cortex.VertexRGB(
    *display_ecc, 
    subject,
)

display_B = con_weighted_dispcx(
    subject, 
    np.array(param_B), 
    np.array(param_rsq), 
    cmap='viridis_r', 
    vmin=0, 
    vmax=400
)
cx_B = cortex.VertexRGB(
    *display_B, 
    subject,
)

display_D = con_weighted_dispcx(
    subject, 
    np.array(param_D), 
    np.array(param_rsq), 
    cmap='magma_r', 
    vmin=0, 
    vmax=1000
)
cx_D = cortex.VertexRGB(
    *display_D, 
    subject,
)

display_polarangle = con_weighted_dispcx(
    subject, 
    np.array(param_pa), 
    np.array(param_rsq), 
    cmap='hsv', 
    vmin=-np.pi, 
    vmax=np.pi
)

cx_polarangle = cortex.VertexRGB(
    *display_polarangle, 
    subject, 
)

display_prf_size = con_weighted_dispcx(
    subject, 
    np.array(param_prf_size), 
    np.array(param_rsq), 
    cmap='plasma', 
    vmin=0, 
    vmax=5
)

cx_prf_size = cortex.VertexRGB(
    *display_prf_size, 
    subject, 
)

# # static figures
if static_imgs:
    cortex.quickshow(
        braindata=cx_TRMI,
        with_colorbar=False,
        depth=0.0,
        thick=1,
        height=2048,
        sampler="nearest",
        with_curvature=True,
        with_dropout=False,
        with_labels=True,
    )
    pl.gca().invert_yaxis()
    pl.savefig(
        os.path.join(analysis_result_dir, subject, "ses-1", "figs", "surf", "all_rsq_s.pdf")
    )
    pl.close("all")
    
if web_view:
    print("creating web view")
    for d, nm, depth in zip([rsq_cx.raw, cx_B.raw, cx_D.raw, cx_ecc.raw, cx_polarangle.raw, cx_prf_size.raw, cx_TRMI.raw, cx_viol_tstat.raw], 
                            ["cx_rsq", "cx_B", "cx_D", "cx_ecc", "cx_pa", "cx_prf_size", "cx_TRMI", "cx_viol_tstat"], 
                            [0, 0, 0, 0, 0, 0, 0, 0]):
        ds = cortex.Dataset(**{nm: d})
        handle = cortex.webgl.show(data=ds, recache=True, labels_visible=(), overlays_visible=('rois',))
        file_pattern = "{base}_{view}_{nm}.png"
        time.sleep(20.0)
        # projection parameters
        basic = dict(
            radius=300, depth=depth, specularity=0, unfold=0.5, contrast=0
        )  # projection=['orthographic'],
        # different views available, more views can be added  and the
        # existing list can be removed
        views = dict(
            myview=dict(altitude=94, azimuth=189, pivot=0),
            # lateral=dict(altitude=90.5, azimuth=181, pivot=180),
            # medial=dict(altitude=90.5, azimuth=0, pivot=180),
            # front=dict(altitude=90.5, azimuth=0, pivot=0),
            # back=dict(altitude=90.5, azimuth=181, pivot=0),
            # top=dict(altitude=0, azimuth=180, pivot=0),
            # bottom=dict(altitude=180, azimuth=0, pivot=0),
        )
        # utility functions to set the different views
        prefix = dict(
            altitude="camera.",
            azimuth="camera.",
            pivot="surface.{subject}.",
            radius="camera.",
            unfold="surface.{subject}.",
            depth="surface.{subject}.",
            specularity="surface.{subject}.",
            contrast="surface.curvature.",
        )
        _tolists = lambda p: {prefix[k] + k: [v] for k, v in p.items()}
        _combine = lambda a, b: (lambda c: [c, c.update(b)][0])(dict(a))
        # Save images by iterating over the different views and surfaces
        for view, vparams in views.items():
            # Combine basic, view, and surface parameters
            params = _combine(basic, vparams)
            # Set the view
            handle._set_view(**_tolists(params))
            # Save image
            if "viol" in nm or "TRMI" in nm:
                filename = file_pattern.format(base=subject+f"_{surprise_analysis_type}", view=view, nm=nm)
            else:
                filename = file_pattern.format(base=subject, view=view, nm=nm)

            output_path = os.path.join(
                figure_result_dir, filename
            )
            handle.getImage(output_path, size=(3840, 2160))
            # the block below trims the edges of the image:
            # wait for image to be written
            while not os.path.exists(output_path):
                pass
            time.sleep(1.5)
            # try:
            #     import subprocess
            #     subprocess.call(['convert', '-trim', output_path, output_path])
            # except:
            #     pass
        # Close the window!
        handle.close()

/Users/dionysus/anaconda3/envs/prfexpectpub/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:4859: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


creating web view
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 37511
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 38719
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 41772
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 15510
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 31652
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 39580
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 27418
Unknown parameter surface.curvature.contrast!
Stop

Stopping server
